In [ ]:
# general imports
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
from matplotlib.colors import ListedColormap
import seaborn as sns

# from our documents
import information as infor
import data_cleaning_functions as dcf
import feature_engineering_functions as fef

# from Scikit Learn library
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import random

#seed for random processes
seed = 42
np.random.seed(seed)
random.seed(seed)


The following pipeline aims to automatize the preprocessing of the stream data from time series.

We will assume that we already have part of the dataset, that will be used as historical data to fill missing values or study seasonalities and stationarities.

This will particular show what to do whenever new data arrive to the system.

# Initial setup

In [8]:
file_path = 'smart_app_data.pkl'
df = pd.read_pickle(file_path)

In [ ]:

# Ensure 'time' column is in datetime format
df['time'] = pd.to_datetime(df['time'])

# Sort the dataframe by 'time'
df = df.sort_values(by='time')

# Define the cutoff date for historical and incoming data
cutoff_date = pd.Timestamp('2024-10-01')

# Split data into historical (before October 2024) and incoming (after October 2024)
# Define the conditions correctly using & and | operators and parentheses for grouping
historical_data = df[(df['time'].dt.year < 2024) | ((df['time'].dt.year == 2024) & (df['time'].dt.month < 10))]
incoming_data = df[(df['time'].dt.year > 2024) | ((df['time'].dt.year == 2024) & (df['time'].dt.month >= 10))]

# Check the first few rows of each dataset to confirm
print("Historical Data (before October 2024):")
print(historical_data.head())

print("\nIncoming Data (after October 2024):")
print(incoming_data.head())

In [14]:
# Check the lengths to confirm the split
print(f"\nNumber of rows in historical data: {len(historical_data)}")
print(f"Number of rows in incoming data: {len(incoming_data)}")

# Check first and last rows of historical and incoming data
print(f"First row in historical data: {historical_data.iloc[0]['time']}")
print(f"Last row in historical data: {historical_data.iloc[-1]['time']}")

print(f"\nFirst row in incoming data: {incoming_data.iloc[0]['time']}")
print(f"Last row in incoming data: {incoming_data.iloc[-1]['time']}")



Number of rows in historical data: 47936
Number of rows in incoming data: 4256
First row in historical data: 2024-03-01 00:00:00+00:00
Last row in historical data: 2024-09-30 00:00:00+00:00

First row in incoming data: 2024-10-01 00:00:00+00:00
Last row in incoming data: 2024-10-19 00:00:00+00:00


In [15]:
# Divide data by machine and kpis (time series)
# Retrieve unique machines and KPIs from the DataFrame
machines = df['name'].unique().tolist()

print("All machines are: ")
print(machines)

kpis = df['kpi'].unique().tolist()

print("All KPIs are: ")
print(kpis)

All machines are: 
['Large Capacity Cutting Machine 1', 'Assembly Machine 2', 'Laser Cutter', 'Riveting Machine', 'Assembly Machine 3', 'Medium Capacity Cutting Machine 1', 'Medium Capacity Cutting Machine 2', 'Low Capacity Cutting Machine 1', 'Testing Machine 3', 'Medium Capacity Cutting Machine 3', 'Testing Machine 1', 'Testing Machine 2', 'Laser Welding Machine 1', 'Laser Welding Machine 2', 'Large Capacity Cutting Machine 2', 'Assembly Machine 1']
All KPIs are: 
['working_time', 'consumption_idle', 'consumption_working', 'power', 'consumption', 'offline_time', 'idle_time', 'cost', 'cost_working', 'cost_idle', 'cycles', 'good_cycles', 'bad_cycles', 'average_cycle_time']


In [ ]:
# Dictionary about specific preprocessing information for kpis
# So the kwargs that will be used in the pipelines are stored there
# The idea is to implement it somewhere else and importing it

# Create the structure for storing batch and counter for each kpi that is produced in the company.
info = {}

for m in list(info.machine.keys()):
    info_asset = {}  # Reset for each machine
    for id in list(infor.machine[m]):
        info_kpi = {}  # Reset for each asset
        for k in list(infor.kpi.keys()):
            info_kpi[k] = [[[], [], [], [], []], [0, 0, 0, 0, 0]]  # One space for the batch and the other for the counter
        info_asset[id] = info_kpi  # Associate KPIs with the asset ID
    info[m] = info_asset  # Associate assets with the machine type
    

# At the end we have a nested dictionary where the atomic element can be retrieved as:
# info['machine_type']['asset_id']['kpi'] --> [batch, counter]
# The batch is a list of 5 sub-lists each containing the batch corrisponding to a specific features

In [ ]:
if x:
    x=x[0]
    nan_cons_thr=3
    im=infoManager(x)
    
    

# Preprocessing pipeline

The pipeline will receive as an input the new incoming data for a specific machine and kpi. Also, to perform the preprocessing it need also to receive the batch of a fixed amount of past data and the information about how specifically handle that kpi for that machine (given by kwargs). 

In [ ]:
def cleaning_pipeline(batch, new_input, kwargs):

    ####### Preprocessing of the data

    # Resampling (if needed)
    if kwargs.get('resample', False):
        new_input= pf.resample_data(batch, new_input, kwargs) #not implemented yet

    # Smoothing (if needed, based on kwargs)
    if kwargs.get('smooth', False):
        new_input = pf.smooth_data(batch, new_input, kwargs) #not implemented yet

    ### DATA CLEANING

    if x:
        x=x[0]
        nan_cons_thr=3
        im=infoManager(x)

        # Try imputation with mean or the HWES model.
        for f in features:
            batch = im.get_batch(f)
            if pd.isna(x[f]):
                    counter=im.update_counter(f)
                    x[f]=predict_missing(batch)
            else: 
                counter=im.update_counter(f, True)
            if counter>nan_cons_thr:
                point_id='/'.join(map(str, list(x.values())[1:4]+list([f])))
                print("It's been " + str(counter) + ' days that [' + str(point_id) + '] is missing')
        print(f'after imputation dp: {x}')

        # Check again the consistency of features and the range.
        if check_f_consistency(x) and check_range(x)[1]:
            pass
        else:  # It means that the imputed data point has not passed the check on the features and on their expected range.
            # In this case we use the LVCF as a method of imputation since it ensures the respect of these conditiono (the last point in the batch has been preiovusly checked)
            for f in features:
                batch=im.get_batch(f)
                x[f]=batch[-1]

        print(f'after check again dp: {x}')
        
        # In the end update batches with the new data point
        for f in features:
            print(f'original batch {f}: {im.get_batch(f)}')
            im.update_batch(f, x[f])
            print(f'batch after update {f}: {im.get_batch(f)}')


    ### FEATURE ENGINEERING
    new_batch = pd.concat([batch, new_input]).sort_values(by='timestamp') 
    # Remove the oldest row (the first row after sorting)
    new_batch = new_batch.iloc[1:]

    return new_batch, new_input


In [ ]:
def feature_engineering_pipeline(dataframe, kwargs):
    ## Check stationarity 
    # (output is False if not stationary, True if it is, None if test couldn't be applied)
    is_stationary = pf.adf_test(dataframe.dropna()) 
    
    ## Check seasonality
    # (output: period of the seasonality None if no seasonalaty was detected.
    seasonality_period = pf.detect_seasonality_acf(dataframe)

    #further check in the case the seasonality pattern is complex and cannot be detected
    if seasonality_period == None:
        # (output: period of the seasonality None if no seasonalaty was detected.
        seasonality_period = pf.detect_seasonality_fft(dataframe)
    
    # (output: the decomposed time series in a list, of form [trend, seasonal, residual],
    # None if it isn't sufficient data or if some error occurs.
    decompositions = pf.seasonal_additive_decomposition(dataframe, seasonality_period) 

    #Make data stationary / Detrend / Deseasonalize (if needed)
    
    make_stationary = kwargs.get('need_stationarity', False)  # Set default to False if not provided
    detrend = kwargs.get('detrend', False) 
    deseasonalize = kwargs.get('deseasonalize', False) 
    get_residuals = kwargs.get('get_residuals', False) 
    
    if make_stationary and not is_stationary:
        dataframe = pf.make_stationary_decomp(dataframe, decompositions)
        is_stationary = pf.adf_test(dataframe.dropna())
        if not is_stationary:
            dataframe = pf.make_stationary_diff(dataframe)

    if detrend:
        if decompositions != None:
            dataframe = pf.rest_trend(dataframe, decompositions)
        else:
            dataframe = pf.make_stationary_diff(dataframe)

    if deseasonalize:
        if decompositions != None:
            dataframe = pf.rest_seasonality(dataframe, decompositions)
        else:
            dataframe = pf.make_stationary_diff(dataframe,seasonality_period=[7]) #default weekly
    
    if get_residuals:
        if decompositions != None:
            dataframe = pf.get_residuals(dataframe, decompositions)
        else:
            dataframe = pf.make_stationary_diff(dataframe)
            dataframe = pf.make_stationary_diff(dataframe,seasonality_period=[7]) #default weekly
            dataframe = pf.make_stationary_diff(dataframe,seasonality_period=[30]) #default monthly

    # Normalize data and apply encoding (if needed)
    categorical_columns = dataframe.select_dtypes(include=['object']).columns

    # Determine if encoder or scaler should be applied based on kwargs
    encoder = kwargs.get('encoder', False)  # Set default to False if not provided
    scaler = kwargs.get('scaler', True)  # Set default to True if not provided
    
    # Apply transformations based on encoder and scaler flags
    if not encoder:  # Only scaler
        transformers = [('num', RobustScaler(), ~dataframe.columns.isin(categorical_columns))]
    elif not scaler:  # Only encoder
        transformers = [('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)]
    else:  # Both encoder and scaler
        transformers = [
            ('num', RobustScaler(), ~dataframe.columns.isin(categorical_columns)),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns),
        ]
    
    preprocessor = ColumnTransformer(transformers=transformers)

    # Fit and transform the data
    dataframe_transformed = preprocessor.fit_transform(dataframe)

    return dataframe_transformed

For the ML algorithms part, should we make a function specific for the drift (separated from the ML function)?

Should we make two different functions for training and usage or is better to have a singular function?

In [ ]:
def features_extraction_pipeline(time_series, kwargs):
    
    ## Definition of metrics for goodness of model 

    ## Drift detection algorithms

    ### ML ALGORTIHMS (this should be divided on training phase and prediction phase)

    ## Check Outliers 

    ## Feature selection

    ## Parameters setting

    ## Feature extraction algorithm (maybe more than one, include selection)

    ## Models comparison (if needed)
        
    return extracted_features #as new KPIs
    

In [ ]:
def forecasting_pipeline(time_series, kwargs):
    
    ## Definition of metrics for goodness of model 

    ## Drift detection algorithms

    ### ML ALGORTIHMS (this should be divided on training phase and prediction phase)

    ## Check Outliers 

    ## Feature selection

    ## Parameters setting

    ## Forecasting algorithm (maybe more than one, include selection)

    ## Models comparison (if needed)
        
    return forecasted_feature #forecasting prediction as a new KPI

In [ ]:
def anomalies_detection_pipeline(time_series, kwargs):
    
    ## Definition of metrics for goodness of model 

    ## Drift detection algorithms

    ### ML ALGORTIHMS (this should be divided on training phase and prediction phase)

    ## Check Outliers 

    ## Feature selection

    ## Parameters setting

    ## Anomalies detection algorithm (maybe more than one, include selection)

    ## Models comparison (if needed)
        
    return anomalies_detection_feature #detected anomaly as a new KPI

# Implementation

## Historical data

### Preprocessing historical data

### Training on historical data

## Incoming data